# Practical Assignment — Machine learning on graphs

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import balanced_accuracy_score, mean_squared_error, roc_curve, auc
from sklearn.datasets import make_moons
from sklearn.neighbors import kneighbors_graph
from IPython.display import clear_output
from tqdm.notebook import tqdm
import requests

In [ ]:
url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_ml_on_graphs/569720_ego_pokec.gml'
open('569720_ego_pokec.gml', 'wb').write(requests.get(url).content)

url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_ml_on_graphs/musae_facebook_ego_802.gml'
open('musae_facebook_ego_802.gml', 'wb').write(requests.get(url).content)

url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_ml_on_graphs/polblogs.gml'
open('polblogs.gml', 'wb').write(requests.get(url).content)

url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_ml_on_graphs/email-Eu-core-temporal.txt'
open('email-Eu-core-temporal.txt', 'wb').write(requests.get(url).content)


### Task 1. Assortativity analysis (1 point)

If the structure of the network is known but the labels of the nodes are hidden, we would like to select a small subset of nodes such that, if we knew their labels, we could accurately predict the labels of all the other nodes. However, it makes sence if labels depend of network structure. Let us remind that assortative mixing in networks is the tendency for nodes to be connected to other nodes that are like (or unlike) them in some way. Assortativity coefficient bounded by

$$-1 \leq r \leq 1$$

where $r \to -1$ means that nodes tend to connect to nodes of the another class, $r \to 1$ — to the same class. Therefore, randomly mixed networks show $r \to 0$ for binary and numeric features, and $r < 0$ for categorical features. 

First, let us check assortativity coefficient for some networks and try to understand which labels can be predicted via network structure.

Write a function `assortativity_coefficients` that takes a graph, an optional list of categorical (or binary) features, an optional list of numerical features and returns a dictionary where a key is a feature name and value is assoratitvity coefficient. _Use `nx.attribute_assortativity_coefficient` and `nx.numeric_assortativity_coefficient`._

In [ ]:
def assortativity_coefficients(G, categorical=[], numerical=[]):
    # YOUR CODE HERE
    raise NotImplementedError()

Here is a subgraph of Slovakian online social network [Pokec](http://snap.stanford.edu/data/soc-Pokec.html)

In [ ]:
G = nx.read_gml('569720_ego_pokec.gml')
coef = assortativity_coefficients(
    G, ['public', 'gender', 'region'], ['age', 'completion_percentage'])
assert len(coef) == 5
assert round(sum(coef.values()), 4) == 0.2832

In [ ]:
pd.DataFrame(coef, index=['assortativity']).T.round(2)

* `public` is 1 if a person publishes his list of friends, and 0 otherwise
* `gender` is 1 for male and 0 for female
* `region` is a region of residence
* `age` is integer age
* `completion_percentage` is a percentage of completion information about a person

Next, look at a network of [political blogosphere in the 2004 US Election](http://www-personal.umich.edu/~mejn/netdata/)

In [ ]:
G = nx.read_gml('polblogs.gml')
coef = assortativity_coefficients(G, ['leaning', 'source'])
pd.DataFrame(coef, index=['assortativity']).T.round(2)

Political blogs leaning are divided into liberal and conservative. Also there is a `source` where this information taken from.

Next, look at subgraph of the [Facebook large page-page network](http://snap.stanford.edu/data/facebook-large-page-page-network.html) restricted to pages from 4 categories which are defined by Facebook. These categories are: politicians, governmental organizations, television shows and companies.

In [ ]:
G = nx.read_gml('musae_facebook_ego_802.gml')
coef = round(assortativity_coefficients(G, ['value'])['value'], 2)
print('category:', coef)

Another way to check numeric features is to calculate correlation of nodes attributes by edges. Let us again loook at attribute `age` in Pokec network. Let us draw a scatterplot with edges where $x$ is an age of the first node and $y$ is an age of the second node in each edge.

Write a function `age_by_edges` that takes Pokec network and returns a tuple of two np.arrays:
* age of the first node in edge
* age of the second node in edge

Size of each array is the number of edges.

In [ ]:
def age_by_edges(G):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = nx.read_gml('569720_ego_pokec.gml')
x, y = age_by_edges(G)
corrcoef = np.corrcoef(x, y)[0, 1]
assert round(corrcoef, 4) == 0.2422

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(x, y, s=10)
plt.title('Correlation : {:.2f}'.format(corrcoef))
plt.xlabel('age')
plt.ylabel('age')
plt.show()

Also it could be useful to transform a graph into bipartite and try to find a stronger dependency. For example, let us look at attribute `age` in edges that connect nodes with opposite `gender`.

Write a function `age_by_gender` that takes Pokec network and returns a tuple of two np.arrays:
* age of a male node in edge (`gender = 1`)
* age of a female node in edge

Size of each array is the number of edges that connect nodes with opposite `gender`.

In [ ]:
def age_by_gender(G):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x, y = age_by_gender(G)
corrcoef = np.corrcoef(x, y)[0, 1]
assert round(corrcoef, 4) == 0.2214

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(x, y, s=10)
plt.title('Correlation : {:.2f}'.format(corrcoef))
plt.xlabel('age, male')
plt.ylabel('age, female')
plt.show()

Now we can find and drop outliers to increase correlation. For example, there are two significant outliers in the area $y>100$.

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(x[y < 100], y[y < 100], s=10)
plt.title(
    'Correlation : {:.2f}'.format(np.corrcoef(x[y < 100], y[y < 100])[0, 1])
)
plt.xlabel('age, male')
plt.ylabel('age, female')
plt.show()

### Task 2. Relational Neighbor Classifier (1 points)

Now let us start again with the facebook dataset and try to predict a page category (0.78 assortativity coefficient).

In [ ]:
G = nx.read_gml('musae_facebook_ego_802.gml')
G = nx.convert_node_labels_to_integers(G)

Prepare train and test sets of nodes to classification. Let us randomly select 30% of nodes as a train set.

In [ ]:
np.random.seed(0)
train_nodes = np.random.choice(G, size=int(0.3 * len(G)), replace=False)
test_nodes = np.array(list(set(G.nodes).difference(train_nodes)))

values = np.array(list(nx.get_node_attributes(G, 'value').values()))
y_train_temp = values[train_nodes]
y_test_temp = values[test_nodes]

print(y_train_temp[:10])

Convert labels into integers for simplicity

In [ ]:
unique = list(set(values))
y_train = np.array([unique.index(val) for val in y_train_temp])
y_test = np.array([unique.index(val) for val in y_test_temp])
print(y_train[:10])

Let us denote $y_i$ as label of node $i$. Relational Neighbor Classifier based on a simple iterative procedure

$$P(y_i = c|\mathcal N(i)) = \frac{1}{Z}\sum_{j \in \mathcal N(i)}A_{ij}P(y_j = c|\mathcal N(j))$$

where $Z$ is a normalizing constant, $\mathcal N(i)$ is neighbors of node $i$. Note that this approuch based on an assumption of strong assortativity: nodes related to each other are similar and likely belong to the same class. The algorithm is:

1. Set an initial conditional distribution $\Phi_0$. Train nodes have a probability one in truth class and zeros in others. Test nodes have an equal probability of each class.
2. Update $\Phi$ only for test nodes by the equation above
3. Repeat 2 until converges: $\Vert \Phi_{i+1} - \Phi_i \Vert < \varepsilon$
4. Predictions are labels with maximal probability

There is a function `relational_neighbor` that predicts labels. Parameters are:
* `G`: graph
* `threshold`: convergence threshold
* `y_train`: np.array, labels for train nodes
* `train_nodes`: np.array, train nodes
* `test_nodes`: np.array, test nodes

The function returns a np.array with labels for test nodes and np.array of norms of a distributions difference in each step before convergence.

In [ ]:
def relational_neighbor(G, threshold, y_train, train_nodes, test_nodes):
    conditional = initial_conditional(G, y_train, train_nodes, test_nodes)
    diffs = []
    while True:

        next_conditional = update_conditional(G, conditional, test_nodes)
        diff = np.linalg.norm(conditional[test_nodes] - next_conditional[test_nodes])
        if diff < threshold:
            break
        diffs.append(diff)
        conditional = next_conditional
        
        if len(diffs) > 1:
            plt.plot(diffs)
            plt.xlabel('Step')
            plt.ylabel('Difference')
            plt.title('Convergence')
            plt.show()
            clear_output(wait=True)
        
    return np.argmax(conditional[test_nodes], axis=1), diffs

Write a function `initial_conditional` that returns np.array with initial conditional distribution where $i$-th row represents probability of belonging of node $i$ to each class. Parameters are the same.

In [ ]:
def initial_conditional(G, y_train, train_nodes, test_nodes):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
conditional = initial_conditional(G, y_train, train_nodes, test_nodes)
assert conditional.shape == (3873, 4)
assert np.all(conditional.sum(axis=1) == 1)
assert np.all(conditional[test_nodes] == 0.25)
assert set(np.unique(conditional[train_nodes])) == {0, 1}

Write a function `update_conditional` that updates and returns np.array with conditional distribution.

In [ ]:
def update_conditional(G, conditional, test_nodes):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
conditional = update_conditional(G, conditional, test_nodes)
assert conditional.shape == (3873, 4)
assert np.all(conditional.sum(axis=1).round(4) == 1)
assert set(np.unique(conditional[train_nodes])) == {0, 1}

An animation of convergence of the algorithm

In [ ]:
y_pred, diffs = relational_neighbor(G, 0.001, y_train, train_nodes, test_nodes)
score = balanced_accuracy_score(y_test, y_pred)
assert len(diffs) < 40
assert score > 0.7

In [ ]:
print('Balanced accuracy:', round(score, 4))

For comparison, a random guess is

In [ ]:
score = balanced_accuracy_score(
    y_test, 
    np.random.choice(range(4), size=len(test_nodes), replace=True))
print('Balanced accuracy:', round(score, 2))

### Task 3. Label propagation by random walks (2 points)

Consider the label propagation algorithm on an artificial dataset consisting of 3 sinusoids with noise.

In [ ]:
N = 600
np.random.seed(0)
x_space = np.linspace(0, 3 * np.pi, int(N/3))
x1 = x_space + np.random.normal(0, 0.2, x_space.shape[0])
y1 = np.sin(x_space) + np.random.normal(0, 0.2, x_space.shape[0])
x2 = x_space + np.random.normal(0, 0.2, x_space.shape[0])
y2 = np.sin(x_space) + np.random.normal(0, 0.2, x_space.shape[0]) - 1.3
x3 = x_space + np.random.normal(0, 0.2, x_space.shape[0])
y3 = np.sin(x_space) + np.random.normal(0, 0.2, x_space.shape[0]) - 2.6

In [ ]:
data_points = np.stack([np.concatenate([x1, x2, x3]), 
                        np.concatenate([y1, y2, y3])], axis=1)
plt.scatter(data_points[:, 0], data_points[:, 1], 
            c=np.repeat(['tab:red', 'tab:orange', 'tab:green'], 200));

Build a graph of k-neighbors of the data points.

In [ ]:
A = kneighbors_graph(data_points, n_neighbors=8)
G = nx.Graph(A)
pos = {i:loc for i, loc in enumerate(data_points)}

Select 20 random train nodes. The goal is to predict an index of the sinusoid for other nodes.

In [ ]:
np.random.seed(0)
train_nodes = np.random.choice(G, size=20, replace=False)
test_nodes = np.array(list(set(range(N)).difference(train_nodes)))

In [ ]:
labels = np.array([0] * 200 + [1] * 200 + [2] * 200)
y_train = labels[train_nodes]
y_test = labels[test_nodes]

Draw the graph where train nodes are highlighted by colors with respect to a label.

In [ ]:
node_color = np.ones((len(G), 3)) * 0.9
node_color[train_nodes[y_train == 0]] = plt.cm.tab10(3)[:3]
node_color[train_nodes[y_train == 1]] = plt.cm.tab10(1)[:3]
node_color[train_nodes[y_train == 2]] = plt.cm.tab10(2)[:3]
nx.draw(G, pos, node_size=50, width=0.5, linewidths=0.3, 
        edgecolors='black', node_color=node_color)

Label propagation method is also assume that closer data points tend to have similar class labels. Let us denote $Y$ as given label matrix, whose $i$-th row representing the label probability distribution of node $i$. Initialization of rows corresponding to unlabeled data points is not important, but let it be a uniform distribution. The algorithm is
1. Propagate $Y \leftarrow PY$ where $P$ is a transition matrix
2. Recover rows of $Y$ corresponding to labeled data points
3. Row-normalize $Y$ to maintain probability interpretation
4. Repeat 1-3 until $Y$ converges
5. Make a prediction as the most likely labels 

Here is a function `label_propagation` that returns predicted labels. Parameters are the same as in the previuos task.

In [ ]:
def label_propagation(G, threshold, y_train, train_nodes, test_nodes):
    Y = initital_Y(G, y_train, train_nodes, test_nodes)
    P = transition_matrix(G)
    while True:
        nextY = update_Y(P, Y, y_train, train_nodes, test_nodes)
        if np.linalg.norm(nextY - Y) < threshold:
            break
        Y = nextY
    y_pred = np.argmax(Y, axis=1)[test_nodes]
    return y_pred

Write a function `initital_Y` that returns np.array with initial label matrix. Parameters are the same.

In [ ]:
def initital_Y(G, y_train, train_nodes, test_nodes):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
Y = initital_Y(G, y_train, train_nodes, test_nodes)
assert Y.shape == (len(G), len(set(y_train)))
assert np.all(Y.sum(axis=1) == 1)
assert Y[train_nodes].max() == 1
assert Y[train_nodes].min() == 0

Write a function `transition_matrix` that returns np.array with transition matrix.

In [ ]:
def transition_matrix(G):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
P = transition_matrix(G)
assert P.shape == (len(G), len(G))
assert np.all(P.sum(axis=1).round(4) == 1)

Write a function `update_Y` that returns np.array with updated label matrix.

In [ ]:
def update_Y(P, Y, y_train, train_nodes, test_nodes):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
nextY = update_Y(P, Y, y_train, train_nodes, test_nodes)
assert nextY.shape == (len(G), len(set(y_train)))
assert np.all(nextY.sum(axis=1).round(4) == 1)
assert nextY[train_nodes].max() == 1
assert nextY[train_nodes].min() == 0

y_pred = label_propagation(G, 0.001, y_train, train_nodes, test_nodes)
assert balanced_accuracy_score(y_test, y_pred) > 0.93

In [ ]:
node_color[test_nodes[y_pred == 0]] = plt.cm.tab10(3)[:3]
node_color[test_nodes[y_pred == 1]] = plt.cm.tab10(1)[:3]
node_color[test_nodes[y_pred == 2]] = plt.cm.tab10(2)[:3]
nx.draw(G, pos, node_size=50, width=0.5, linewidths=0.3, 
        edgecolors='black', node_color=node_color)

### Task 4. Tikhonov regularization on graphs (2 points)

Consider node regression with Tikhonov ($L_2$, Ridge) regularization on an artificial dataset that was again converted into graph by k-neighbors.

In [ ]:
N = 600
data_points, labels = make_moons(n_samples=N, noise=0.15, random_state=0)
A = kneighbors_graph(data_points, n_neighbors=5).toarray()
G = nx.Graph(A)
pos = {i:loc for i, loc in enumerate(data_points)}

In [ ]:
np.random.seed(0)
train_nodes = np.random.choice(G, size=20, replace=False)
test_nodes = np.array(list(set(range(N)).difference(train_nodes)))
labels[labels == 0] = 10
labels[labels == 1] = 100
y_train = labels[train_nodes]
y_test = labels[test_nodes]

Here are labels of blue nodes equal to 10, reds equal to 100. Other labels are unknown.

In [ ]:
node_color = np.ones((len(G), 3)) * 0.9
node_color[train_nodes[y_train == 10]] = plt.cm.coolwarm(0)[:3]
node_color[train_nodes[y_train == 100]] = plt.cm.coolwarm(255)[:3]
nx.draw(G, pos, node_size=50, width=0.5, linewidths=0.3, 
        edgecolors='black', node_color=node_color)

Let $\mathbf y$ be labels of nodes and $\tilde y_i = y_i - \frac{1}{k}\sum_{j=1}^k y_j$ where $k$ is the number of known labels. Unknown labels are given as zeros $\tilde y_i = 0$. The objective is to minimize the square loss function plus the smoothness penalty

$$\mathbf{\tilde f} = \text{argmin}_\mathbf{f}\left(\frac{1}{k}\sum_{i=1}^k(f_i - \tilde y_i)^2 + \gamma \mathbf{f}^T L \mathbf{f}\right)$$

where $L$ is a graph Laplacian. The analytical solution is given as

$$\mathbf{\tilde f} = (k \gamma L + I)^{-1}\mathbf{\tilde y}$$

where $I$ is a diagonal matrix with ones and zeros. $I_{ii} = 1$ if a label of $i$-th node is known.

_Remark: for the stability of the algorithm, we use $\tilde y_i$ instead of $y_i$._

Write a function `tikhonov_regularization` that takes the same parameters as above (`gamma` is a coefficient of regularization), and returns predicted labels.

In [ ]:
def tikhonov_regularization(G, gamma, y_train, train_nodes, test_nodes):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
y_pred = tikhonov_regularization(G, 0.001, y_train, train_nodes, test_nodes)
assert mean_squared_error(y_test, y_pred) < 282

In [ ]:
plt.figure(figsize=(16, 5))

for i, gamma in enumerate([0.01, 0.5]):
    y_pred = tikhonov_regularization(G, gamma, y_train, train_nodes, test_nodes)
    node_color = labels
    node_color[test_nodes] = y_pred
    plt.subplot(1, 2, i+1)
    nx.draw(G, pos, node_size=50, width=0.5, linewidths=0.3, 
            node_color=node_color, edgecolors='black', cmap=plt.cm.coolwarm)
    plt.title('gamma: {}'.format(gamma))

### Task 5. Similarity based link prediction (2 points)

Consider link prediction on the [e-mails network](http://snap.stanford.edu/data/email-Eu-core-temporal.html) where nodes are members of a research institution and edges are e-mails given with timestamps. The goal is to predict occurrence of edges in the test time period using information from the train time period only.

In [ ]:
email_df = pd.read_csv(
    'email-Eu-core-temporal.txt', 
    delimiter=' ', 
    names=['sender', 'receiver', 'timestamp']
)
email_df.head()

Next, consider the following preprocessing procedure:
1. Select edges by given train and test time periods, for example, [0, 1000) is train and [1000, 2000) is test
2. Build a _core_ — a network where every edge occurs at least $k_\text{train}$ times in the train time period and at least $k_\text{test}$ times in the test time period. Let the core be undirected, so occurrences edges (1, 0) and (0, 1) are computed together.
3. From the core, select a train set of edges $E_\text{train}$ that occur for the first time in the train period. The same for $E_\text{test}$. Consider first occurrances in the train and test periods only.

Write a function `train_test_edges` that takes a pd.DataFrame `email_df` with e-mail network, a tuple with the train time period borders `train_period`, say, (0, 1000), a similar tuple `test_period`, the number of edges occurrences `ktrain` and `ktest`. The function returns two lists with tuples — train and test edges. Every edge is returned of the form where the first node is less than the second, for example [(1, 2), (2, 3)] is ok, but [(2, 1), (3, 2)] is wrong.

In [ ]:
def train_test_edges(email_df, train_period, test_period, ktrain, ktest):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
train_edges, test_edges = train_test_edges(email_df, (1e7, 2e7), (2e7, 2.5e7), 3, 3)
_train_edges, _test_edges = np.array(train_edges), np.array(test_edges)
assert np.all(_train_edges[:, 0] < _train_edges[:, 1])
assert np.all(_test_edges[:, 0] < _test_edges[:, 1])
assert len(set(train_edges).intersection(test_edges)) == 0
assert _train_edges.shape == (4147, 2)
assert _test_edges.shape == (418, 2)

The similarity based algorithm:
1. Compute similarity matrix for all pairs of nodes except $E_\text{train}$
2. Order that pairs in descending of similarity
3. Select some threshold and predict links for all pairs above the threshold

Write a function `sim_link_prediction` that takes a list with train edges and test edges. The function predicts links and returns a tuple with metrics: 
* two np.arrays: FPR (false positive rate) and TPR (true positive rate) in descending of thresholds obtained by Jaccard coefficient, `nx.jaccard_coefficient`
* the same, by Adamic/Adar index, `nx.adamic_adar_index`
* the same, by resource allocation index, `nx.resource_allocation_index`

_Hint: use `sklearn.metrics.roc_curve`._

In [ ]:
def sim_link_prediction(train_edges, test_edges):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
jac, adam, res = sim_link_prediction(train_edges, test_edges)

In [ ]:
assert round(auc(jac[0], jac[1]), 4) == 0.8371
assert round(auc(adam[0], adam[1]), 4) == 0.8500
assert round(auc(res[0], res[1]), 4) == 0.8495

Let us look at ROC AUC curve to compare similaritites.

In [ ]:
cases = [[jac[0], jac[1], 'Jaccard'], 
         [adam[0], adam[1], 'Adamic/Adar'], 
         [res[0], res[1], 'Resource alloc.']]
for fpr, tpr, label in cases:
    plt.plot(fpr, tpr, lw=2, 
             label='{}, AUC={:.4f}'.format(label, auc(fpr, tpr)))
plt.plot([0, 1], [0, 1], lw=2, linestyle='--', label='Random, AUC=0.5')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC AUC')
plt.legend()
plt.show()

### Task 6. Node2Vec graph embedding, DeepWalk (2 points)

DeepWalk is an approach for learning latent representations of nodes in a network. These latent representations encode social relations in a continuous vector space, which is easily exploited by statistical models. The algorithm is divided into two steps:
1. Generate random walks
2. Encode a matrix of random walks into low-dimensional space

Write a function `random_walks` that takes a graph, number of random walks `n_walks` starting from every node, path length of walk `path_length`. The function returns a np.array of a shape `(n_walks, path_length)` where every row is a single random path.

In [ ]:
def random_walks(G, n_walks, path_length):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(0)
G = nx.karate_club_graph()
walks = random_walks(G, 10, 100)

assert walks.shape == (340, 100)
for i, j in zip(walks[0, :-1], walks[0, 1:]):
    assert G.has_edge(i, j)
assert np.all(walks[:, 0] == np.repeat(np.arange(34), 10))

Now let us apply Word2Vec model for embedding nodes into 2D space. 

Write a function `encode` that takes a np.array with random walks matrix and returns positions of nodes — a dictionary where keys are nodes and values are np.arrays with 2D embedding.

_Hints:_
* _Convert the random walks matrix from np.array into list with string entries_
* _Create a model `gensim.models.word2vec.Word2Vec` with `size=2`, try to adjust `alpha`, `hs`, `iter` parameters_
* _`model.wv` contains embedding 2D vectors_

In [ ]:
def encode(walks):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
pos = encode(walks)
assert type(pos) == dict
assert len(pos) == 34
assert pos[0].shape == (2,)
assert np.linalg.norm(pos[0] - pos[6]) < np.linalg.norm(pos[0] - pos[32])
assert np.linalg.norm(pos[1] - pos[2]) < np.linalg.norm(pos[1] - pos[32])

In [ ]:
plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
nx.draw(G, pos, node_size=100, width=0.3, linewidths=1, 
        node_color='tab:orange', edgecolors='black')
plt.title('DeepWalk')
plt.subplot(1, 2, 2)
nx.draw_random(G, node_size=100, width=0.3, linewidths=1, 
        node_color='tab:orange', edgecolors='black')
plt.title('Random')
plt.show()